<a href="https://colab.research.google.com/github/drscook/redistricting_wrangler/blob/main/redistricting_2025_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Advances in Data Engineering for Redistricting

Scott Cook

Tarleton State Univ

2025-10-13

#Background

##Geography

Consider [Geographic Hierarchy](https://www2.census.gov/geo/pdfs/reference/geodiagram.pdf). We need data from 4 sources that live at different levels:
- 2020 Decennial Census PL94-171
    - level: block (only dataset at this smallest level)
    - https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html
    - For convenience, we're actually pulling from TX Legislative Council: https://data.capitol.texas.gov/dataset/2020-census-geography    
- American Community Surveys (ACS)
    - level: some block group, some tract
    - https://www.census.gov/programs-surveys/acs/data.html
    - Like an annual mini-Census
    - Use 5 year estimates (more stable)
    
- Citizen Voting Age Population (CVAP) special tabulation
    - level: block group
    - https://www.census.gov/programs-surveys/acs/data.html
    - Not a part of decennial census or ACS
- Elections
    - level: [voting tabulation district (VTD)](https://data.capitol.texas.gov/dataset/vtds)
    - https://data.capitol.texas.gov/dataset/comprehensive-election-datasets-compressed-format

##Problem

While Census-controlled levels are nested, VTDs only respect blocks. In other words, a block lies entirely within exactly one VTD. But a block group or tract might be split across mutliple VTDs (and conversely).

##Solution

Push all data down to common refinement (pieces that do not cross any boundary) then aggregate back up to desired level
1. Fetch raw data from relevant sources
1. Geospatially intersect (overlay) block and VTD geometries to create *pieces* (essentially blocks with slight correction for geospatial misalignments)
1. Apportion data to pieces proportional to voting age population from 2020 Census (recall 2020 Census is the only data at block level)


##Links

- Census
    - API Key: https://api.census.gov/data/key_signup.html
    - Geographic Hierarchy: https://www2.census.gov/geo/pdfs/reference/geodiagram.pdf
    - Geoids: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
    - American Community Survey (ACS): https://www.census.gov/programs-surveys/acs/data.html
    - Citizen Voting Age Population (CVAP): https://www.census.gov/programs-surveys/decennial-census/about/voting-rights/cvap.html
    - Block shapefiles: https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/
    - Python: https://pypi.org/project/census/

- Texas Legislative Council
    - Voting Tabulation Districts (VTD): https://data.capitol.texas.gov/dataset/vtds
    - Elections: https://data.capitol.texas.gov/dataset/comprehensive-election-datasets-compressed-format
    - PL94-171: https://data.capitol.texas.gov/dataset/2020-census-geography
    - School Districts (not used here): https://data.capitol.texas.gov/dataset/school-districts

- Other
    - CRS:
        - https://docs.qgis.org/3.40/en/docs/gentle_gis_introduction/coordinate_reference_systems.html
        - https://epsg.io/3085
        - https://epsg.io/4269
    - Tarrant precincts: https://www.tarrantcountytx.gov/en/elections/interactive-maps/commissioner-precinct-maps.html

- Quesion: Did Andrea find block assignment files for on TX Lege data portal? If so, where? Census has BAF for congressional, senate, and house here, but I think she has a source with more (school districts, judicial, etc).



#Setup

1. Request Census API key
    1. https://api.census.gov/data/key_signup.html
    1. path into api_key = '___' below
1. Mount google drive
    1. click folder icon on left (under key icon)
    1. click folder iocn with inset google drive triangle
    1. allow
    1. new folder "drive" appears in file browser (refresh if not)
    1. open it and navigate to where you want to save
    1. hover over it, click 3 dots on right, click "copy path"
    1. paste into root = pathlib.Path('___') below
1. Run cell below once at the start of session to install packages

In [ ]:
#run once to begin each session
%pip install -U ipython-autotime Census us
from IPython import get_ipython
get_ipython().kernel.do_shutdown(restart=True)

#Code

In [ ]:
%reload_ext autotime
import pathlib, requests, zipfile, pickle, census, us, pyarrow.parquet as pq
import numpy as np, pandas as pd, geopandas as gpd
root = pathlib.Path('/content/drive/MyDrive/redistricting_2025_10')/'data'
api_key = '5640e76608e24d8d6cc35b96ce35028445957cb5'
session = census.Census(api_key)
state = us.states.TX
tx_lege_year = 2024

#################### helpers ####################
def prep(df):
    return df.convert_dtypes().rename(columns=lambda x: x.strip().lower().replace(' ','_').replace('-','_'))


def dump(file, obj, **kwargs):
    """write obj to file"""
    file.parent.mkdir(parents=True, exist_ok=True)  #make parent directory
    #write using method associated to file.suffix
    if file.suffix == '.parquet':
        prep(obj).to_parquet(file, **kwargs)
    elif file.suffix in ['.csv', '.txt']:
        prep(obj).to_csv(file, **kwargs)
    else:
        with open(file, 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL, **kwargs)
    return obj


def load(file, **kwargs):
    """read obj from file"""
    if file.suffix == '.parquet':
        if any(col.type=='binary' for col in pq.ParquetFile(file).schema_arrow):  #if parquet contains geometry columns, use geopandas
            return prep(gpd.read_parquet(file, **kwargs))
        else:
            return prep(pd.read_parquet(file, **kwargs))
    elif file.suffix in ['.csv', '.txt']:
        return prep(pd.read_csv(file, **kwargs))
    else:
        try:
            return prep(gpd.read_file(file, **kwargs))  #try geopandas to test is file is geospatial (like shapefile)
        except:
            with open(file, 'rb') as f:
                return pickle.load(f, **kwargs)


def fetch(file, url, unzip=True):
    """fetch data from url and write to file"""
    if not file.exists():
        print(f'fetching {url} to {file}')
        response = requests.get(url)  #request url
        assert response.ok
        file.parent.mkdir(parents=True, exist_ok=True)  #make parent directory
        with open(file, 'wb') as f:
            f.write(response.content)  #pull data
        if unzip and zipfile.is_zipfile(file):  #unzip
            with zipfile.ZipFile(file, 'r') as f:
                f.extractall(file.parent)
    return file


geoid_structure = {'state':2, 'county':3, 'tract':6, 'block_group':1, 'block':4}
def make_geoid(df, level):
    """make geoid from separate columns"""
    df['geoid'] =''
    g = lambda col, width: df.pop(col).astype(str).str.rjust(width, '0')  #prepend leading 0's to ensure correct string length
    for col, width in geoid_structure.items():
        df['geoid'] += g(col, width)  #iteratively append to existing geoid
        if col == level:  #stop at level and rename geoid to level
            df[level] = df.pop('geoid').astype('Int64')
            return df


def get_paths(stem):
    path = root/stem
    dst = path/f'{stem.replace('/', '_')}.parquet'
    src = dst.with_suffix('.zip')
    return path, dst, src

#################### data ####################

def get_pl94():
    """fetch & process PL94-171 - only works for TX (currently)"""
    path, dst, src = get_paths(f'pl94/2020/{state.abbr}')
    if not dst.exists():
        fetch(src, f'https://data.capitol.texas.gov/dataset/2b59f5ce-5fa4-4040-a550-caffbe8986c4/resource/33ed77c5-951b-4a88-9de7-9c90c4ba50db/download/blocks_pop.zip')  #download raw data
        repl = {'sctbkey':'block', 'fename':'county'} | {k:f'{k}_2020' for k in ['anglo','asian','hisp','total','vap','black','bh','nanglo','anglovap','hispvap','bhvap','blackvap','asianvap','nanglovap']}
        df = load(src.parent/'Blocks_Pop.txt')[repl.keys()].rename(columns=repl)
        dump(dst, df)
    return load(dst)


def get_vtds():
    """fetch & process voting tabulation districts - only works for TX"""
    path, dst, src = get_paths(f'vtds/{tx_lege_year}/{state.abbr}')
    if not dst.exists():
        fetch(src, f'https://data.capitol.texas.gov/dataset/4d8298d0-d176-4c19-b174-42837027b73e/resource/906f47e4-4e39-4156-b1bd-4969be0b2780/download/vtds_pg.zip', unzip=False)  #download raw data
        df = load(src, columns=['VTDKEY'])
        dump(dst, df)
    return load(dst)


def get_blocks():
    """fetch & process block geometries"""
    path, dst, src = get_paths(f'blocks/2020/{state.abbr}')
    if not dst.exists():
        fetch(src, f'https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_{state.fips}_tabblock20.zip', unzip=False)  #download raw data
        df = load(src, columns=['GEOID20'])
        df['block'] = df.pop('geoid20').astype('Int64')
        dump(dst, df)
    return load(dst)


def get_pieces():
    """intersect geometries into pieces that do not cross any boundary"""
    path, dst, src = get_paths(f'pieces/2024/{state.abbr}')
    if not dst.exists():
        V = get_vtds()
        B = get_blocks().to_crs(V.crs)
        df = gpd.overlay(V, B, keep_geom_type=True)  #intersect
        df['area'] = df.area  #compute areas
        #according to Texas Legislative Council, vtds are specifically created so each block is wholly contained in exactly 1 vtd
        #however, there can be tiny errors due to precision issues, so this discards all but the largest piece from any block
        #then we merge 2020 Census data from PL94-171
        df = df.loc[df['area']==df.groupby('block')['area'].transform('max')]
        df['block_group'] = df['block']//1000
        df['tract'] = df['block']//10000
        #we can add more districts like congressional, senate, house, school district, etc by joining block equivalency files here
        df = df.merge(get_pl94())  #merge pl94 data
        dump(dst, df[[*df.columns.drop('geometry'), 'geometry']])  #move geometry column to end for convenience
    return load(dst).set_index(['block','block_group','tract','vtdkey'])  #include additional districts here too


def get_multipliers(level):
    """compute multipliers to apportion from level to blocks via vap_2020 from PL94-171"""
    path, dst, src = get_paths(f'multipliers/2020/{state.abbr}/{level}')
    if not dst.exists():
        P = get_pieces()
        #for each block, compute its vap_2020 / total vap_2020 in the {level} that contains it
        T = P.groupby(level)['vap_2020'].transform('sum').clip(1)  #vap_202 in the {level}; clip(1) replaces 0->1 to avoid 0/0 below
        df = (P['vap_2020']/T).rename('multiplier').to_frame()
        dump(dst, df)
    return load(dst)


def apportion(data, level):
    """apportion data from level to blocks"""
    data = data.set_index(level)
    df = get_multipliers(level).join(data)  #join multiplier to data
    df *= df.pop('multiplier').to_frame().values  #pop multiplier and use it to multiply all remaining columns
    # check that we recover the original values by aggregating back to original level
    chk = data - df.groupby(level).sum()
    assert np.max(np.abs(chk) < 0.001)
    return df.squeeze()


def get_acs_yr_code(yr, code):
    """fetch & process 1 acs variable for 1 year"""
    path, dst, src = get_paths(f'acs/{yr}/{state.abbr}/{code}')
    if not dst.exists():
        print(f'fetching {dst}')
        #detect whether this acs variable is available at block_group level; if not settle for tracts
        test = session.acs5.state_county_blockgroup(code, state.fips, '003', '*', year=yr)  #pull 1 county at block_group level
        level = 'block_group' if any(t[code] is not None for t in test) else 'tract'  #any t[code] is not None <=> available for block_group,
        #fetch from census api using their python package at the level determined above
        if level == 'block_group':
            raw = session.acs5.state_county_blockgroup(code, state.fips, '*', '*', year=yr)
        else:
            raw = session.acs5.state_county_tract(code, state.fips, '*', '*', year=yr)
        df = make_geoid(prep(pd.DataFrame(raw)), level)
        dump(dst, df)
    data = load(dst)
    level = data.columns[-1]
    return apportion(data, level)


def get_acs_yr(yr, acs_variables):
    """fetch & process all acs variables for 1 year"""
    L = [get_acs_yr_code(yr, code).rename(f'{alias}_{yr}') for code, alias in acs_variables]
    return pd.concat(L, axis=1)


def get_cvap_yr(yr):
    """fetch & process cvap for 1 year"""
    path, dst, src = get_paths(f'cvap/{yr}')
    level = 'block_group'
    if not dst.exists():
        fetch(src, f'https://www2.census.gov/programs-surveys/decennial/rdo/datasets/{yr}/{yr}-cvap/CVAP_{yr-4}-{yr}_ACS_csv_files.zip')  #download raw data
        df = load(src.parent/'BlockGr.csv', encoding='latin1')  #load relevant file
        df[level] = df['geoid'].str[-12:].astype('Int64')  #create block_group identifier columns
        df = df.pivot_table(index=level, columns='lntitle', values=['cit_est','cvap_est'], fill_value=0)  #pivot long to wide so each row is a blkgrp with colunms for each citizen & cvap variable
        df.columns = [f'{k[:-4]}_{v}_{yr}' for k,v in df.columns]  #clean up after pivot
        dump(dst, df.reset_index())
    data = load(dst)
    return apportion(data, level)


def get_elections(offices, start=2010, end=2030):
    """get election results - only works for TX"""
    path, dst, src = get_paths(f'elections/{tx_lege_year}/{state.abbr}')
    level = 'vtdkey'
    if not dst.exists():
        fetch(src, f'https://data.capitol.texas.gov/dataset/35b16aee-0bb0-4866-b1ec-859f1f044241/resource/e1cd6332-6a7a-4c78-ad2a-852268f6c7a2/download/general-vtds-election-data.zip')  #download raw data
        L = [load(file, usecols=['vtdkeyvalue','Office','Name','Party','Votes']).assign(year=int(file.stem[:4])) for file in src.parent.iterdir() if 'General_Election_Returns' in file.stem and 'City' not in file.stem]
        df = pd.concat(L).rename(columns={'vtdkeyvalue':level})
        for k in ['office','name']:
            df[k] = df[k].str.replace('_',' ').str.replace('.', '')  #process strings
        dump(dst, df)
    data = (
        load(dst)
        .query(f'office in @offices and party in ["D","R"] and year >= {start} and year <= {end}', engine='python')  # keep elections for specified offices in specified years for democrats and republicans
        .assign(candidate=lambda X: X['office']+'_'+X['name']+'_'+X['party']+'_'+X['year'].astype('string'))  # concat information as column names for pivot
        .pivot_table(index=level, columns='candidate', values='votes', fill_value=0)  # pivot long to wide so each row is a vtd with colunms for each election
        .rename_axis(columns=None).reset_index()  # clean up after pivot
        )
    return apportion(data, level)


acs_variables = [
    ['B01001_001E' , 'pop_total'],
    ['B01001I_001E', 'hisp_total'],
]
offices = ['President',
           'US Sen',
           'Governor',
           'Lt Governor',
           'Attorney Gen',
]
years = [2020,2021,2022,2023]
A = {yr: get_acs_yr(yr, acs_variables) for yr in years}
C = {yr: get_cvap_yr(yr) for yr in years}
E = get_elections(offices, start=min(years))